<a href="https://colab.research.google.com/github/joostnjns/Google-Earth-Engine/blob/master/SVM_2000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
var cost_svm = 70;

// Query satellite image within bounding box
var aoi = ee.Geometry.Rectangle([14.35, 26.95, 14.51, 27.12]);

var collection00 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2000-01-01', '2000-02-10').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image00 = collection00.sort('CLOUD_COVER').first();

var collection05 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2005-01-01', '2005-02-20').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image05 = collection05.sort('CLOUD_COVER').first();

var collection10 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2010-01-01', '2010-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
var image10 = collection10.sort('CLOUD_COVER').first();

function calcMSAVI2(image){
  var msavi2 = image.expression(
  '((2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2)*1000',  //to be in same scale as spectral bands 
  {
    'NIR': image.select('B4'), 
    'RED': image.select('B3')
  }
  );
  image = image.addBands([msavi2]);
  image = image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','constant']).rename(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','msavi2']);
  return image
}

var image00 = calcMSAVI2(image00);
var image05 = calcMSAVI2(image05);
var image10 = calcMSAVI2(image10);





// Define the visualization parameters.
var vizParams1 = {
  bands: ['B3','B2','B1'],
  min: 0,
  max: 5000,
  gamma: [0.9,1.1,1]
};

// Visualize satellite image              
Map.setCenter(14.42, 27.04, 13);

Map.addLayer(image00.clip(aoi), vizParams1, 'true00');
//Map.addLayer(image05.clip(aoi), vizParams1, 'true05');
//Map.addLayer(image10.clip(aoi), vizParams1, 'true10');


//////////////////
//   TRAINING   //
//////////////////

// Load training data and sample ground truth polygons
var trainData = ee.FeatureCollection('users/jneujens/trainData2000');    // Load training data
//Map.addLayer(trainData);

var label = 'classname';     // Property containing land cover labels
var groundTrue00 = image00.sampleRegions({     // Get training from polygons on image
  collection: trainData,
  properties: ['classvalue'],
  scale: 30
});

var groundTrue05 = image05.sampleRegions({     // Get training from polygons on image
  collection: trainData,
  properties: ['classvalue'],
  scale: 30
});

var groundTrue10 = image10.sampleRegions({     // Get training from polygons on image
  collection: trainData,
  properties: ['classvalue'],
  scale: 30
});

// Split ground truth into training and test data
//var withRandom = groundTrue.randomColumn('random');
//var split = 0.99;  // Roughly 70% training, 30% testing.
//var training = withRandom.filter(ee.Filter.lt('random', split));
//var testing = withRandom.filter(ee.Filter.gte('random', split));

// Create an SVM classifier with custom parameters.
var classifier00 = ee.Classifier.svm({
  decisionProcedure: 'Voting',
  svmType: 'C_SVC',
  kernelType: 'LINEAR',
  cost: 0.1,

});

var classifier05 = ee.Classifier.svm({
  decisionProcedure: 'Voting',
  svmType: 'C_SVC',
  kernelType: 'LINEAR',
  cost: 10
});

var classifier10 = ee.Classifier.svm({
  decisionProcedure: 'Voting',
  svmType: 'C_SVC',
  kernelType: 'LINEAR',
  cost: 50
});

// Train the classifier.
var trained00 = classifier00.train(groundTrue00, 'classvalue', ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'msavi2']);
//var trained05 = classifier05.train(groundTrue05, 'classvalue', ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'msavi2']);
//var trained10 = classifier10.train(groundTrue10, 'classvalue', ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'msavi2']);


// Display results

var sebha00 = image00.clip(aoi).classify(trained00);
//var sebha05 = image05.clip(aoi).classify(trained05);
//var sebha10 = image10.clip(aoi).classify(trained10);

var vizParams2 = {min: 0, max: 2, palette: ['grey', 'green','red']}

// Display result

Map.addLayer(sebha00, vizParams2, 'class00');
//Map.addLayer(sebha05, vizParams2, 'class05');
//Map.addLayer(sebha10, vizParams2, 'class10');


// Get confusion matrix
